In [ ]:
# !pip install pynq

In [ ]:
# Move the overlays to the working directory
# After running the code, modify line 75 to 'self.<kernel_name>'
# import shutil

# overlay_location="/home/csl/anaconda3/envs/hls4ml-u55c/lib/python3.10/site-packages/hls4ml/templates/vivado_accelerator/alveo/python_drivers/axi_stream_driver.py"
# shutil.copyfile(overlay_location,'axi_stream_driver.py')

'axi_stream_driver.py'

In [1]:
import os

# Set the environment variables for XRT
xrt_213_path = "/home/csl/yeop/XRT/build/Debug/opt/xilinx/xrt"
os.environ["XILINX_XRT"] = xrt_213_path
os.environ["LD_LIBRARY_PATH"] = f"{xrt_213_path}/lib:" + os.getenv("LD_LIBRARY_PATH", "")
os.environ["PATH"] = f"{xrt_213_path}/bin:" + os.getenv("PATH", "")

In [2]:
!xbutil --version

Version              : 2.13.0
Branch               : 2022.1
Hash                 : ca4bdf101cd1d698377f5a7113d8583f709c2143
Hash Date            : 2025-02-27 06:03:19
XOCL                 : 2.13.466, f5505e402c2ca1ffe45eb6d3a9399b23a0dc8776
XCLMGMT              : 2.13.466, f5505e402c2ca1ffe45eb6d3a9399b23a0dc8776


In [ ]:
# in terminal
# sudo chown root:xrt /dev/xclmgmt55296
# sudo chmod 666 /dev/xclmgmt55296
# sudo chmod 666 /dev/dri/renderD128

In [3]:
!ls -l /dev/xclmgmt55296
!ls -l /dev/dri/renderD128

crw-rw-rw- 1 root xrt 498, 0 Apr 18 09:04 /dev/xclmgmt55296
crw-rw-rw-+ 1 root render 226, 128 Apr 18 09:04 /dev/dri/renderD128


In [4]:
!xbutil examine -d 0000:d8:00.1 -r thermal


-----------------------------------------------------
1/1 [0000:d8:00.1] : xilinx_u55c_gen3x16_xdma_base_3
-----------------------------------------------------
Thermals
  Temperature            : Celcius
  PCB Top Front          :     49 C
  PCB Top Rear           :     51 C
  Cage0                  :     49 C
  FPGA                   :     61 C
  Int Vcc                :     57 C
  FPGA HBM               :     56 C



In [5]:
import numpy as np

from pynq import allocate
from pynq import Overlay
from pynq import Device

from matplotlib import pyplot as plt
from datetime import datetime

/home/csl/anaconda3/envs/hls4ml-u55c/lib/python3.10/site-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)
/home/csl/anaconda3/envs/hls4ml-u55c/lib/python3.10/site-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


In [6]:
# Load the test data (just random data for now)
X_test = np.random.rand(1000, 10)
y_test = np.random.randint(2, size=(1000, 1))

In [7]:
try:
    device = Device.active_device
    print(f"Device found: {device}")
except RuntimeError as e:
    print(f"Error: {e}")

Device found: <pynq.pl_server.xrt_device.XrtDevice object at 0x7fcf85254dc0>


In [8]:
from axi_stream_driver import NeuralNetworkOverlay

# If error -> "source /opt/xilinx/xrt/setup.sh" in terminal
# Load the xclbin file to the board
ol = NeuralNetworkOverlay("./hls4ml_prj_u55c_new/xclbin_files/myproject_kernel.xclbin")

In [ ]:
# Check the name of the kernel
ol?

In [ ]:
ol.myproject_1?

In [10]:
y_pred_fpga, _, rate = ol.predict(X_test, y_test.shape, np.int16, profile=True)

RuntimeError: Allocate failed: 4294967277

In [54]:
for i in range(5):
    true_label = y_test[i]
    pred_label = y_pred_fpga[i]
    print(f"Sample {i}: True label = {true_label}, Predicted = {pred_label}")

Sample 0: True label = [1], Predicted = [508]
Sample 1: True label = [0], Predicted = [0]
Sample 2: True label = [1], Predicted = [0]
Sample 3: True label = [1], Predicted = [0]
Sample 4: True label = [1], Predicted = [0]


In [90]:
from sklearn.metrics import accuracy_score

print("Accuracy (FPGA):", accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred_fpga, axis=1)))

Accuracy (FPGA): 1.0


In [91]:
sensors = Device.active_device.sensors

for name, sensor in sensors.items():
    print(name, sensor.power)

0v85 None
12v_aux Sensor {name=12v_aux_power, value=0.0W}
12v_pex Sensor {name=12v_pex_power, value=0.0W}
12v_sw None
1v8 None
3v3_aux None
3v3_pex None
mgt0v9avcc None
mgtavtt None
sys_5v5 None
vccint Sensor {name=vccint_power, value=0.0W}


In [9]:
# Record power consumption

from pynq.pmbus import DataRecorder
from pynq import Device

# Setup
sensors = Device.active_device.sensors
recorder = DataRecorder(sensors["12v_aux"].power,
                        sensors["12v_pex"].power,
                        sensors["vccint"].power)

# Start recording just before calling ol.predict
recorder.record(0.1)

# Run your kernel (this is where power spikes will happen)
y_pred_fpga, _, rate = ol.predict(X_test, y_test.shape, np.int16, profile=True)

# Stop recording right after
recorder.stop()

# View the recorded power data
frame = recorder.frame
print(frame.head())

Classified 1000 samples in 0.0011769999999999999 seconds (849617.6720475787 inferences / s)
Or 1.1769999999999998 us / inferences
                            Mark  12v_aux_power  12v_pex_power  vccint_power
2025-04-19 04:58:57.263910   0.0       5.736544       11.92244           0.0
